# Create Neural Network for Image Inversion with numpy

## I. Read training data

In [2]:
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [3]:
# Convert PIL image to array normalised to 0 to 1 rage.
max_color = 255
def image_2_array(filename):
    pil_img=Image.open(filename)
    pil_arr=np.array(pil_img)
    pil_scaled_arr = (pil_arr / max_color)
    return pil_img, pil_scaled_arr

# Invert grayscale PIL image
def invert_array(arr):
    inverted_arr = 1 - arr
    return inverted_arr
    
# Convert normalized array to PIL image
def array_2_image(arr):
    arr_scaled = (arr) * max_color
    im = Image.fromarray(arr_scaled)
    return im

In [4]:
# Read training images into array of shape (m_samples, imsize, imsize, 1)
train_data_onedimension = np.array([])
train_labl_onedimension = np.array([])
DIR="imgset20x20x1000"
imsize=20
orig_pattern = DIR + "/origin*"
invr_pattern = DIR + "/inverse*"
import glob
for f in glob.glob(orig_pattern):
    img,a = image_2_array(f)
    train_data_onedimension = np.append(train_data_onedimension,a)
for f in glob.glob(invr_pattern):
    img,a = image_2_array(f)
    train_labl_onedimension = np.append(train_labl_onedimension,a)

print train_data_onedimension.shape
print train_labl_onedimension.shape

(400000,)
(400000,)


In [5]:
m_samples = train_data_onedimension.shape[0] / imsize / imsize
train_data = train_data_onedimension.reshape(m_samples, imsize, imsize, 1)
train_labl = train_labl_onedimension.reshape(m_samples, imsize, imsize, 1)
print train_data.shape
print train_labl.shape


(1000, 20, 20, 1)
(1000, 20, 20, 1)


In [6]:
# Split data between training and test sets 700 : 300
TEST_SIZE = 300
TRAINING_SIZE = m_samples - TEST_SIZE

test_data  = train_data[:TEST_SIZE,:,:,:]
train_data = train_data[TEST_SIZE:,:,:,:]

test_labl  = train_labl[:TEST_SIZE,:,:,:]
train_labl = train_labl[TEST_SIZE:,:,:,:]
print("Dataset dimensions:")
print train_data.shape
print train_labl.shape
print test_data.shape
print test_labl.shape
diff_matr = train_data[1]-train_labl[1]
print diff_matr[:2,:2,:]

Dataset dimensions:
(700, 20, 20, 1)
(700, 20, 20, 1)
(300, 20, 20, 1)
(300, 20, 20, 1)
[[[ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]]]


## II. Initialise weights

In [53]:
# layer sizes
s = np.array([imsize*imsize, imsize*imsize])
W = np.random.random((s[0], s[1]))

In [54]:
print W.shape
print W

(400, 400)
[[ 0.75426934  0.8940074   0.93419127 ...,  0.70054067  0.37729373
   0.54581379]
 [ 0.00557844  0.62377065  0.94055771 ...,  0.05683272  0.43310855
   0.44195954]
 [ 0.79952494  0.37228256  0.58126998 ...,  0.6488431   0.2360349
   0.37852085]
 ..., 
 [ 0.17615692  0.29341372  0.11020371 ...,  0.42576221  0.29389477
   0.84066644]
 [ 0.69710923  0.55459072  0.36785511 ...,  0.46996209  0.24499483
   0.07216449]
 [ 0.42183989  0.41787727  0.69709838 ...,  0.65477391  0.01687452
   0.34774023]]


## III. Cost function and hypothesis

$J = \frac{1}{2m} \sum\limits_{i=1}^{m}\sum\limits_{k=1}^{imsize*imsize} (h_{i_k} - y_{i_k})^2 + \frac{\lambda}{2m} \sum\limits_{j=1}^{imsize*imsize} \sum\limits_{k=1}^{imsize*imsize}(W_{jk})^2$

$h = g(Wx)$

In [55]:
def loss(h, y, W, lambda_):
    m = h.shape[0]  # Number of samples
    J = sum(sum((h - y)*(h - y))) / (2 * m) 
    J = J + sum(sum (W * W)) * lambda_ / (2 * m)
    return J

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def hypothesis(x, W):
    y = sigmoid(np.dot(x, W))
    return y

## IV. Training

In [65]:
iterations = 1
lambda_ = 0.0001
x = np.array(train_data.reshape(train_data.shape[0],imsize*imsize))
y = np.array(train_labl.reshape(train_labl.shape[0],imsize*imsize))
print x.shape
Dlt = np.zeros(s[0],s[1])
for iter in xrange(iterations):
    # Forward propagation
    h = hypothesis(x, W)
    J = loss(h, y, W, lambda_)
    print J
    # Backward propagation
    print "delta"
    l1_delta = y - h
    l0_delta = np.dot(l1_delta,W)* h * (1 - h)
    print l1_delta.shape
    print l0_delta.shape
    #Dlt = 
    # Update weights
    #W = W + 

(700, 400)
50.0038159489
delta
(700, 400)
(700, 400)


In [18]:
# Test array operations
a = np.array([[ 1., 2.], [2., 3.]])
b = np.array([[ 2., 2.], [2., 2.]])

In [45]:
print(a*b)
print ".."
print(a*a)
print(sum(a*a))
print"-----"
print(sum((b - a)*(b - a)))

[[ 2.  4.]
 [ 4.  6.]]
..
[[ 1.  4.]
 [ 4.  9.]]
[  5.  13.]
-----
[ 1.  1.]
